In [ ]:
!pip install --upgrade tensorflow


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.16.1


In [ ]:
# Get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# Load and prepare data
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'text'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])


--2024-06-17 14:15:17--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.2’

train-data.tsv.2    100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2024-06-17 14:15:17 (7.91 MB/s) - ‘train-data.tsv.2’ saved [358233/358233]

--2024-06-17 14:15:17--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.2’

valid-data.tsv.2    100%[====

In [ ]:
# Encode labels
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})


In [ ]:
# Tokenize text
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['text'])
train_sequences = tokenizer.texts_to_sequences(train_df['text'])
test_sequences = tokenizer.texts_to_sequences(test_df['text'])


In [ ]:
# Pad sequences
max_length = 100
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')


In [ ]:
# Create model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),  # Adding a Dropout layer for regularization
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model with a slightly adjusted learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # You can experiment with learning rates
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
num_epochs = 20  # Increasing the number of epochs
history = model.fit(train_padded, train_df['label'], epochs=num_epochs, validation_data=(test_padded, test_df['label']), verbose=2)


Epoch 1/20
131/131 - 4s - 27ms/step - accuracy: 0.8634 - loss: 0.3963 - val_accuracy: 0.8657 - val_loss: 0.3667
Epoch 2/20
131/131 - 1s - 4ms/step - accuracy: 0.8660 - loss: 0.3654 - val_accuracy: 0.8657 - val_loss: 0.3596
Epoch 3/20
131/131 - 1s - 4ms/step - accuracy: 0.8660 - loss: 0.3607 - val_accuracy: 0.8657 - val_loss: 0.3519
Epoch 4/20
131/131 - 1s - 5ms/step - accuracy: 0.8660 - loss: 0.3471 - val_accuracy: 0.8657 - val_loss: 0.3378
Epoch 5/20
131/131 - 1s - 4ms/step - accuracy: 0.8660 - loss: 0.3210 - val_accuracy: 0.8657 - val_loss: 0.2991
Epoch 6/20
131/131 - 1s - 5ms/step - accuracy: 0.8672 - loss: 0.2701 - val_accuracy: 0.8815 - val_loss: 0.2345
Epoch 7/20
131/131 - 1s - 4ms/step - accuracy: 0.9160 - loss: 0.1960 - val_accuracy: 0.9540 - val_loss: 0.1559
Epoch 8/20
131/131 - 1s - 5ms/step - accuracy: 0.9679 - loss: 0.1181 - val_accuracy: 0.9691 - val_loss: 0.0945
Epoch 9/20
131/131 - 1s - 8ms/step - accuracy: 0.9809 - loss: 0.0726 - val_accuracy: 0.9763 - val_loss: 0.0730


In [ ]:
# Function to predict messages based on model
def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(padded)[0][0]
    label = 'spam' if prediction > 0.5 else 'ham'
    return [float(prediction), label]

In [ ]:
# Function to predict messages based on model
def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(padded)[0][0]
    label = 'spam' if prediction > 0.5 else 'ham'
    return [float(prediction), label]

# Test the function and model
def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"]
    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True
    actual_predictions = []

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        actual_predictions.append(prediction[1])
        print(f"Message: {msg}")
        print(f"Expected: {ans}, Predicted: {prediction[1]}, Confidence: {prediction[0]}")
        if prediction[1] != ans:
            passed = False

    print("\nExpected Answers:", test_answers)
    print("Actual Predictions:", actual_predictions)

    if set(test_answers) == set(actual_predictions):
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Message: how are you doing today
Expected: ham, Predicted: ham, Confidence: 0.0005458322702907026
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Message: sale today! to stop texts call 98912460324
Expected: spam, Predicted: ham, Confidence: 0.2430655062198639
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Message: i dont want to go. can we try it a different day? available sat
Expected: ham, Predicted: ham, Confidence: 0.00021974458650220186
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Message: our new mobile video service is live. just install on your phone to start watching.
Expected: spam, Predicted: spam, Confidence: 0.9540842771530151
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Message: you have won £1000 cash! call to claim your prize.
Expected: spam, Predicted: spam, Confidence: 0.9898502230644226
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Message: i'll bring it tomorrow. don't forget the milk.
Expected: ham, Predicted: ham, Confidence: 0.00011692034604493529
1/1 ━━━━━━━━━━━━━